In [1]:
import os
import math
import pickle
import torch
from torch import optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from google.colab import drive
from random import sample

drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
!pip install transformers sentencepiece

In [ ]:
# !pip install transformers sentencepiece

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 初始化 MBART 模型和分词器，并移到设备上
model_name = 'facebook/mbart-large-50-many-to-one-mmt'
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)

def translate_text(text, source_lang="zh_CN", target_lang="en_XX"):
    tokenizer.src_lang = source_lang
    # 确保编码后的张量也在正确的设备上
    encoded = tokenizer(text, return_tensors="pt").to(device)
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id[target_lang])
    return tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

def process_and_translate_srt(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as infile, open(output_path, 'w', encoding='utf-8') as outfile:
        srt_block = []
        for line in infile:
            if line.strip():
                srt_block.append(line)
            else:
                if srt_block:
                    # 处理并翻译srt_block
                    for srt_line in srt_block:
                        outfile.write(srt_line)
                        if '-->' not in srt_line and not srt_line.strip().isdigit():
                            # 翻译并写入翻译后的文本
                            translated_line = translate_text(srt_line.strip())
                            print(translated_line)
                            outfile.write(translated_line + '\n')
                    outfile.write('\n')
                    srt_block = []
        # 处理文件最后一个块
        if srt_block:
            for srt_line in srt_block:
                outfile.write(srt_line)
                if '-->' not in srt_line and not srt_line.strip().isdigit():
                    translated_line = translate_text(srt_line.strip())
                    outfile.write(translated_line + '\n')
            outfile.write('\n')

input_srt_path = '/content/drive/MyDrive/users/yangyi/Moive/Bilingual.srt'
output_srt_path = '/content/drive/MyDrive/users/yangyi/Moive/Bilingual_Translated.srt'

process_and_translate_srt(input_srt_path, output_srt_path)